In [1]:
import os
import pandas as pd

In [2]:
os.chdir('/Users/arnorfati/Documents/Data/엘포인트')

In [3]:
os.getcwd()

'/Users/arnorfati/Documents/Data/엘포인트'

# 데이터 로드

In [4]:
cust_demo = pd.read_csv('01.고객DEMO.txt')
shp_sale = pd.read_csv('02.쇼핑업종 상품구매.txt')
unshp_sale = pd.read_csv('03.쇼핑외 업종 상품구매.txt')
shp_nm = pd.read_csv('04.쇼핑업종 상품분류.txt')

In [5]:
cust_demo.columns = ['고객번호','성별','연령대','거주지']
shp_sale.columns = ['고객번호','영수증번호','업종','상품_소분류','점포코드','구매일자','구매시간','구매금액','구매수량']
unshp_sale.columns = ['고객번호','업종','이용월','이용금액','이용건수']
shp_nm.columns = ['업종','상품_소분류','상품_소분류명','상품_중분류명','상품_대분류명']

## 고객데이터 EDA

In [6]:
print('#########  dim 확인  #########')
print(cust_demo.shape)
print('#########  data type 확인  #########')
print(cust_demo.dtypes)
cust_demo.head()

#########  dim 확인  #########
(20000, 4)
#########  data type 확인  #########
고객번호      int64
성별        int64
연령대      object
거주지     float64
dtype: object


,고객번호,성별,연령대,거주지
0,1,1,60PRD,52.0
1,2,2,60PRD,80.0
2,3,2,60PRD,620.0
3,4,1,60PRD,120.0
4,5,1,60PRD,NaN


In [7]:
# 거주지는 쓸필요 없는 정보니 날리자
cust_demo = cust_demo.drop(['거주지'], axis = 1)
# 고객번호 str 변환
cust_demo['고객번호'] = cust_demo['고객번호'].astype('str')

In [8]:
print('#########  연령대 확인  #########')
print(cust_demo.groupby('연령대')['고객번호'].count())
print('#########  성별 확인  #########')
print(cust_demo.groupby('성별')['고객번호'].count())

#########  연령대 확인  #########
연령대
20PRD    2339
30PRD    5053
40PRD    5183
50PRD    4744
60PRD    2681
Name: 고객번호, dtype: int64
#########  성별 확인  #########
성별
1     7815
2    12185
Name: 고객번호, dtype: int64


## 구매 데이터 EDA

In [9]:
print('#########  dim 확인  #########')
print(shp_sale.shape) # 364만건..
print('#########  data type 확인  #########')
print(shp_sale.dtypes)
shp_sale.head()
# 구매 업종이랑 상품 소분류명을 모르겠다... 정보를 합쳐야 겠음.
# PK설정을 위해 전처리 필요. ==> shp_nm 의 업종, 상품 소분류로 merge 필요.

#########  dim 확인  #########
(3641082, 9)
#########  data type 확인  #########
고객번호       int64
영수증번호      int64
업종        object
상품_소분류     int64
점포코드       int64
구매일자       int64
구매시간       int64
구매금액       int64
구매수량       int64
dtype: object


,고객번호,영수증번호,업종,상품_소분류,점포코드,구매일자,구매시간,구매금액,구매수량
0,4008,2108,A01,215,2,20150216,13,59600,2
1,6379,2109,A01,75,29,20150213,11,35000,1
2,6379,2109,A01,149,4,20150115,10,85000,1
3,8002,2110,A01,138,10,20151220,10,25000,1
4,8002,2110,A01,138,10,20151220,10,21000,1


In [10]:
print('#########  dim 확인  #########')
print(shp_nm.shape) # 3788건의 상품정보 존재
print('#########  data type 확인  #########')
print(shp_nm.dtypes)
shp_nm.head()

#########  dim 확인  #########
(3788, 5)
#########  data type 확인  #########
업종         object
상품_소분류      int64
상품_소분류명    object
상품_중분류명    object
상품_대분류명    object
dtype: object


,업종,상품_소분류,상품_소분류명,상품_중분류명,상품_대분류명
0,A04,341,단행본서적(직배),특수점서적,서적/음반
1,A04,1,삼각김밥,삼각김밥,미반
2,A04,2,The커진삼각김밥,삼각김밥,미반
3,A04,3,말이김밥,김밥,미반
4,A04,4,트레이김밥,김밥,미반


In [11]:
shp_sale['고객번호'] = shp_sale['고객번호'].astype('str')
shp_sale['업종'] = shp_sale['업종'].astype('str')
shp_sale['상품_소분류'] = shp_sale['상품_소분류'].astype('str')
shp_nm['업종'] = shp_nm['업종'].astype('str')
shp_nm['상품_소분류'] = shp_nm['상품_소분류'].astype('str')

In [12]:
shp_sale_master = pd.merge(shp_sale,shp_nm,on = ['업종','상품_소분류'], how = 'left')

In [13]:
shp_sale_master.head()

,고객번호,영수증번호,업종,상품_소분류,점포코드,구매일자,구매시간,구매금액,구매수량,상품_소분류명,상품_중분류명,상품_대분류명
0,4008,2108,A01,215,2,20150216,13,59600,2,이지캐주얼,캐주얼,여성의류
1,6379,2109,A01,75,29,20150213,11,35000,1,건강식품,건강식품,식품
2,6379,2109,A01,149,4,20150115,10,85000,1,기초A,화장품,잡화
3,8002,2110,A01,138,10,20151220,10,25000,1,기타,섬유잡화,잡화
4,8002,2110,A01,138,10,20151220,10,21000,1,기타,섬유잡화,잡화


In [14]:
shp_sale_master[shp_sale_master['고객번호'] == '1'].sort_values(['구매일자','구매시간'])

,고객번호,영수증번호,업종,상품_소분류,점포코드,구매일자,구매시간,구매금액,구매수량,상품_소분류명,상품_중분류명,상품_대분류명
530464,1,100812,A01,236,2,20150128,12,29800,1,홈웨어,기타의류,여성의류
1079289,1,157750,A01,121,2,20150128,13,18000,2,식당가 한식,가공식품,식품
1201670,1,37450,A01,367,2,20150128,13,48000,1,L/C골프의류,골프용품,스포츠
3591428,1,110941,A01,34,2,20150128,13,10000,1,기타생선,수산품,식품
1175531,1,191488,A01,479,2,20150307,12,169000,1,주방잡화,주방용품,가정
1456339,1,222915,A01,2,2,20150307,12,9000,1,돈육,육류,식품
2431732,1,149128,A01,15,2,20150307,12,8900,1,디저트류,차/커피,식품
2245486,1,129205,A01,493,2,20150308,11,547000,1,기능성침대,생활잡화,가정
2512493,1,325566,A01,46,2,20150402,15,13000,1,초밥,가공식품,식품
407302,1,204739,A01,71,2,20150513,12,1000,2,어묵,젓갈/반찬,식품


In [23]:
shp_sale_master.head()

,고객번호,영수증번호,업종,상품_소분류,점포코드,구매일자,구매시간,구매금액,구매수량,상품_소분류명,상품_중분류명,상품_대분류명,상품
0,4008,2108,A01,215,2,20150216,13,59600,2,이지캐주얼,캐주얼,여성의류,여성의류_캐주얼_이지캐주얼
1,6379,2109,A01,75,29,20150213,11,35000,1,건강식품,건강식품,식품,식품_건강식품_건강식품
2,6379,2109,A01,149,4,20150115,10,85000,1,기초A,화장품,잡화,잡화_화장품_기초A
3,8002,2110,A01,138,10,20151220,10,25000,1,기타,섬유잡화,잡화,잡화_섬유잡화_기타
4,8002,2110,A01,138,10,20151220,10,21000,1,기타,섬유잡화,잡화,잡화_섬유잡화_기타


In [17]:
shp_sale_master["상품"] = shp_sale_master["상품_대분류명"] +"_" + shp_sale_master["상품_중분류명"] + "_" + shp_sale_master["상품_소분류명"]

In [22]:
# shp_sale_master.상품[shp_sale_master.업종 == 'B01'] = "호텔"
# shp_sale_master.상품[shp_sale_master.업종 == 'B02'] = "여행사"
# shp_sale_master.상품[shp_sale_master.업종 == 'B03'] = "면세점"
# shp_sale_master.상품[shp_sale_master.업종 == 'C01'] = "영화관"
# shp_sale_master.상품[shp_sale_master.업종 == 'C02'] = "테마파크"
# shp_sale_master.상품[shp_sale_master.업종 == 'C03'] = "야구관람"
# shp_sale_master.상품[shp_sale_master.업종 == 'D01'] = "롯데리아"
# shp_sale_master.상품[shp_sale_master.업종 == 'D02'] = "페밀리레스토랑"
# shp_sale_master.상품[shp_sale_master.업종 == 'D03'] = "엔젤리너스"

In [24]:
all_data_demo2 = shp_sale_master[["고객번호","상품"]]

In [25]:
del shp_sale_master

In [26]:
all_data_demo2.head()

,고객번호,상품
0,4008,여성의류_캐주얼_이지캐주얼
1,6379,식품_건강식품_건강식품
2,6379,잡화_화장품_기초A
3,8002,잡화_섬유잡화_기타
4,8002,잡화_섬유잡화_기타


In [27]:
all_data_demo3 = all_data_demo2.groupby(["고객번호"])["상품"].apply(','.join).reset_index()

In [28]:
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import TfidfModel

In [30]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(sentence.split(",")) # deacc=True removes punctuations

data_words = list(sent_to_words(all_data_demo3.상품))

print(data_words[:1])

[['가정_주방용품_수입주방', '가정_주방용품_수입주방', '스포츠_골프용품_N/B골프의류', '여성의류_란제리/내의_란제리', '식품_젓갈/반찬_어묵', '식품_젓갈/반찬_어묵', '식품_젓갈/반찬_어묵', '식품_젓갈/반찬_어묵', '식품_차/커피_디저트류', '여성의류_기타의류_홈웨어', '식품_가공식품_제과', '제빵', '가전_소형가전_수입청소기 외', '아동_아동_유아복', '아동_아동_유아복', '식품_가공식품_식당가 한식', '가정_주방용품_주방잡화', '스포츠_골프용품_L/C골프의류', '식품_육류_돈육', '스포츠_골프용품_L/C골프의류', '식품_가공식품_식당가 한식', '식품_가공식품_식당가 한식', '스포츠_스포츠_디스커버리', '건강식품_건강선물세트_홍삼선물세트', '식품_젓갈/반찬_어묵', '식품_젓갈/반찬_어묵', '식품_젓갈/반찬_어묵', '식품_육류_우육', '식품_가공식품_식당가 일식', '식품_젓갈/반찬_어묵', '식품_가공식품_에스닉푸드', '아동_아동_유아복', '가정_생활잡화_기능성침대', '식품_차/커피_디저트류', '식품_가공식품_초밥', '식품_농산물_청과', '식품_주류_음료', '식품_농산물_유기농채소', '아동_아동_내의', '식품_차/커피_디저트류', '여성의류_란제리/내의_란제리', '식품_가공식품_초밥', '스포츠_골프용품_N/B골프의류', '식품_가공식품_식당가 중식', '남성의류_셔츠_셔츠단기행사', '스포츠_골프용품_N/B골프의류', '가정_식기_수입도자기', '스포츠_골프용품_N/B골프의류', '남성의류_트래디셔널_수입화', '식품_차/커피_수입식품', '식품_육류_계육', '식품_농산물_유기농채소', '스포츠_스포츠_몬츄라', '식품_가공식품_초밥', '식품_수산품_기타생선', '스포츠_스포츠_L/C 아웃도어']]


In [31]:
# Create Dictionary
id2word = corpora.Dictionary(data_words)

# Create Corpus  
texts = data_words

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 2), (4, 1), (5, 1), (6, 1), (7, 1), (8, 2), (9, 4), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 3), (16, 1), (17, 1), (18, 3), (19, 2), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 8), (26, 1), (27, 3), (28, 1), (29, 1), (30, 3), (31, 1), (32, 2), (33, 1)]]


In [32]:
model = TfidfModel(corpus)  # fit model

In [33]:
tfidf_corpus = []
for i in range(0,len(corpus)-1) :
    tfidf_corpus.append(model[corpus[i]])
    

In [34]:
K = 14
D = len(corpus)
W = len(id2word)
num_topics = 14
iterations = 1000 #iteration
alpha = 50/K #typical(Griffiths and Steyvers (2004) suggest)
eta = 200/W      

In [35]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
id2word=id2word,
num_topics=K,
random_state=100,
update_every=1,
chunksize=100,
passes=10,
alpha=alpha,
eta = eta,
per_word_topics=True)

In [36]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [37]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/Users/arnorfati/anaconda/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
12    -0.136304  0.370513       1        1  15.120040
5      0.240179 -0.035126       2        1  12.349873
2      0.245829  0.009221       3        1   9.219468
13     0.239535 -0.039419       4        1   7.573255
4      0.251853 -0.024281       5        1   7.166932
0     -0.207341 -0.144744       6        1   6.659387
11    -0.112708  0.370422       7        1   5.758494
8     -0.097707  0.037971       8        1   5.724903
7      0.151742 -0.025538       9        1   5.692602
9     -0.212447 -0.153129      10        1   5.560468
1     -0.216869 -0.153864      11        1   5.093291
6      0.213800 -0.031908      12        1   4.997876
10    -0.185509 -0.098184      13        1   4.620609
3     -0.174053 -0.081934      14        1   4.462808, topic_info=     Category          Freq                    Term         Total  loglift  \
term                                                                         
36    Default  49113.000000     여성의류_캐주얼_global SPA  49113.000000  30.0000   
219   Default  43632.000000            주방잡화_균일가_다이소  43632.000000  29.0000   
154   Default  29940.000000           잡화_화장품_기초 화장품  29940.000000  28.0000   
35    Default  26418.000000          식품_가공식품_전문베이커리  26418.000000  27.0000   
374   Default  24092.000000              주류_맥주_국산맥주  24092.000000  26.0000   
685   Default  23034.000000              주류_맥주_수입맥주  23034.000000  25.0000   
223   Default  38519.000000     청소/수납용품_종량제봉투_재사용봉투  38519.000000  24.0000   
20    Default  22941.000000               식품_농산물_청과  22941.000000  23.0000   
53    Default  25935.000000              과자_스낵_일반스낵  25935.000000  22.0000   
28    Default  27260.000000            식품_차/커피_수입식품  27260.000000  21.0000   
15    Default  19477.000000          식품_가공식품_식당가 한식  19477.000000  20.0000   
146   Default  20263.000000             식품_축산가공_유제품  20263.000000  19.0000   
93    Default  18587.000000               식품_농산물_채소  18587.000000  18.0000   
161   Default  28829.000000           가공대용식_라면_봉지라면  28829.000000  17.0000   
224   Default  18664.000000                      커피  18664.000000  16.0000   
19    Default  15433.000000            식품_농산물_유기농채소  15433.000000  15.0000   
25    Default  16187.000000             식품_젓갈/반찬_어묵  16187.000000  14.0000   
91    Default  14732.000000             식품_농산물_농산가공  14732.000000  13.0000   
500   Default  18139.000000             면과자_스낵_일반스낵  18139.000000  12.0000   
60    Default  14553.000000            기타_기타_상품군미지정  14553.000000  11.0000   
208   Default  13468.000000      브랜드화장품_기초화장품_일반화장품  13468.000000  10.0000   
155   Default  12171.000000           잡화_화장품_색조 화장품  12171.000000   9.0000   
34    Default  14224.000000            식품_가공식품_브랜드샵  14224.000000   8.0000   
490   Default  14867.000000         냉동_아이스크림_바아이스크림  14867.000000   7.0000   
87    Default  13358.000000          식품_가공식품_일반가공식품  13358.000000   6.0000   
907   Default  11237.000000        식품_가공식품_지방스넥(부대)  11237.000000   5.0000   
295   Default  21937.000000       생활잡화_청소욕실용품_종량제봉투  21937.000000   4.0000   
137   Default  14601.000000            식품_가공식품_일식델리  14601.000000   3.0000   
531   Default  11928.000000          유음료_가공우유_컵커피우유  11928.000000   2.0000   
39    Default  16479.000000            가공대용식_라면_컵라면  16479.000000   1.0000   
...       ...           ...                     ...           ...      ...   
2186  Topic14   1398.301758     남성의류_트래디셔널_타미힐피거상품군   1398.908447   3.1090   
1671  Topic14   1374.453247          잡화_피혁잡화_4대 B/D   1375.059937   3.1090   
83    Topic14   1185.365356         스포츠_스포츠_뉴발란스아울렛   1185.972046   3.1089   
1933  Topic14   1021.809692       남성의류_트래디셔널_헤지스상품군   1022.416382   3.1088   
317   Topic14   3050.506836        여성의류_캐주얼_온라인스트리트   3052.518066   3.1087   
1433  Topic14    560.913269            식품_가공식품_푸드코트    561.519958   3.1083   
1975  Topic14    480.803375         잡화_화장품_아울렛(화장품)

In [38]:
pyLDAvis.save_html(vis, 'lda.html')

In [42]:
import os
os.getcwd()


'/Users/arnorfati/Documents/Data/엘포인트'

In [26]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=tfidf_corpus,
id2word=id2word,
num_topics=K,
random_state=100,
update_every=1,
chunksize=100,
passes=10,
alpha=alpha,
eta = eta,
per_word_topics=True)

In [27]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
3     -0.041524 -0.307485       1        1  9.488083
8     -0.020519 -0.300050       2        1  8.640168
12     0.209748  0.070756       3        1  8.023399
13     0.211407  0.055622       4        1  7.965565
11     0.101837  0.011845       5        1  7.879636
10     0.221991  0.079745       6        1  7.805220
2     -0.077750 -0.033017       7        1  7.724803
5      0.197093  0.017030       8        1  7.529750
7     -0.200899  0.139368       9        1  6.945667
9     -0.044268  0.089320      10        1  6.134731
1     -0.202163  0.132969      11        1  5.797477
0     -0.166516  0.100105      12        1  5.526327
4     -0.089887 -0.045343      13        1  5.507638
6     -0.098548 -0.010866      14        1  5.031528, topic_info=     Category          Freq                 Term         Total  loglift  \
term                                                                      
8     Default  41262.000000                  면세점  41262.000000  30.0000   
35    Default  40762.000000                  영화관  40762.000000  29.0000   
76    Default  32925.000000                 롯데리아  32925.000000  28.0000   
37    Default  22957.000000                 테마파크  22957.000000  27.0000   
32    Default  25102.000000                엔젤리너스  25102.000000  26.0000   
251   Default  25578.000000  여성의류_캐주얼_global SPA  25578.000000  25.0000   
545   Default  17742.000000        잡화_화장품_기초 화장품  17742.000000  24.0000   
376   Default  14906.000000    생활잡화_가정잡화_생활잡화균일가  14906.000000  23.0000   
242   Default  16680.000000       식품_가공식품_전문베이커리  16680.000000  22.0000   
924   Default  12191.000000        잡화_화장품_색조 화장품  12191.000000  21.0000   
355   Default  12825.000000          면과자_스낵_일반스낵  12825.000000  20.0000   
879   Default  11150.000000           주류_맥주_수입맥주  11150.000000  19.0000   
326   Default  13196.000000           낙농_우유_일반우유  13196.000000  18.0000   
252   Default  10038.000000        여성의류_캐주얼_스트리트  10038.000000  17.0000   
568   Default  16140.000000    생활잡화_청소욕실용품_종량제봉투  16140.000000  16.0000   
16    Default  10208.000000       식품_가공식품_식당가 한식  10208.000000  15.0000   
561   Default  10822.000000      냉동_아이스크림_바아이스크림  10822.000000  14.0000   
170   Default   8752.000000        스포츠_스포츠_스포츠의류   8752.000000  13.0000   
138   Default  21655.000000         주방잡화_균일가_다이소  21655.000000  12.0000   
225   Default  11772.000000   브랜드화장품_기초화장품_일반화장품  11772.000000  11.0000   
29    Default  13260.000000         식품_차/커피_수입식품  13260.000000  10.0000   
455   Default  10780.000000           채소_두채류_두부류  10780.000000   9.0000   
43    Default  10098.000000         가공대용식_라면_컵라면  10098.000000   8.0000   
285   Default  12895.000000           주류_맥주_국산맥주  12895.000000   7.0000   
224   Default   9268.000000   브랜드화장품_기초화장품_여성화장품   9268.000000   6.0000   
50    Default  11746.000000           과자_스낵_일반스낵  11746.000000   5.0000   
11    Default   7740.000000     스포츠_스포츠_L/C 아웃도어   7740.000000   4.0000   
930   Default   8104.000000          가정_문구/팬시_서적   8104.000000   3.0000   
1771  Default   8848.000000       유음료_가공우유_컵커피우유   8848.000000   2.0000   
178   Default  12096.000000       여성의류_캐주얼_이지캐주얼  12096.000000   1.0000   
...       ...           ...                  ...           ...      ...   
395   Topic14   4701.390137  아동유아브랜드_유아브랜드_유아브랜드   4735.248535   2.9823   
964   Topic14   4236.855957       완구_BOYS 1_액션피겨   4270.714355   2.9815   
905   Topic14   4199.635742       스포츠_스포츠_아디다스의류   4233.494141   2.9814   
517   Topic14   4056.094971        아동_아동_L/C 아동복   4089.953613   2.9811   
518   Topic14   3743.158447        아동_아동_N/B 아동복   3777.017090   2.9804   
86    Topic14   3478.294678        스포츠_스포츠_나이키의류   3512.153320   2.9798   
171   Topic14   3466.040527       스포츠브랜드_브랜드_브랜드   3499.899170   2.9797   
821   Topic14   3305.599854        유음료_가공우유_초코우유   3339.458496   2.9793   
583   Topic14   3092.753662       

In [46]:
import numpy as np
a = np.random.randn(4,3)
b = np.random.randn(3,2)
c = a * b

ValueError: operands could not be broadcast together with shapes (4,3) (3,2) 

In [45]:
c.shape

(2, 3)